In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
from scipy import stats

In [2]:
data_path = '../../data/lda_jsd_toprev_conlen/'

In [3]:
fandom_list = [i for i in os.listdir(data_path) if 'tsv' in i]

In [4]:
fandom_list

['shakespare_william_works_temporal_lda_jsd_toprev_full.tsv',
 'buffy_the_vampire_slayer_temporal_lda_jsd_toprev_full.tsv',
 'les_miserables_all_media_types_temporal_lda_jsd_toprev_full.tsv',
 'one_direction_temporal_lda_jsd_toprev_full.tsv',
 'ms_paint_adventures_temporal_lda_jsd_toprev_full.tsv',
 'marvel_temporal_lda_jsd_toprev_full.tsv',
 'kuroko_no_basuke_temporal_lda_jsd_toprev_full.tsv',
 'tolkien_j_r_r_works_&_related_fandoms_temporal_lda_jsd_toprev_full.tsv',
 'harry_potter_temporal_lda_jsd_toprev_full.tsv',
 'haikyuu_temporal_lda_jsd_toprev_full.tsv',
 'dcu_temporal_lda_jsd_toprev_full.tsv',
 'bishoujo_senshi_sailor_moon_temporal_lda_jsd_toprev_full.tsv',
 'the_walking_dead_&_related_fandoms_temporal_lda_jsd_toprev_full.tsv',
 'supernatural_temporal_lda_jsd_toprev_full.tsv',
 'doctor_who_&_related_fandoms_temporal_lda_jsd_toprev_full.tsv',
 'homestuck_temporal_lda_jsd_toprev_full.tsv',
 'arthurian_mythology_&_related_fandoms_temporal_lda_jsd_toprev_full.tsv',
 'attack_on_tita

In [5]:
def bootstrap_resample(li):
    if len(li) > 0:
        ave_original = np.average(li)
        aves = []
        for i in range(1000):
            sample = []
            for i in range(len(li)):
                sample.append(random.choice(li))
            aves.append(np.average(sample))
        tail = sorted(aves)[24]
        head = sorted(aves)[975]
        return (ave_original, tail, head)

## Aggregate

In [6]:
def create_normed_dfs(df):
    
    # Normalize effects of authors by turning kudos into z-score corresponding to works from this author
    # Seems not useful b/c many authors only have 1 work
#     author_list = df.Author.drop_duplicates().tolist()
#     combined = []
#     for author in author_list:
#         df_a = df[df.Author == author]
#         if len(df_a) > 1:
#             df_a['Kudos'] = (df_a['Kudos'] - df_a['Kudos'].mean())/df_a['Kudos'].std(ddof=0)
#             combined.append(df_a)        
#     df = pd.concat(combined)
#     df = df[df.Kudos != 0]
    
#     Normalize by log scale
    df = df.replace([np.inf, -np.inf], np.nan)
    df['Bookmarks'].fillna(0, inplace=True)
    df['Bookmarks'] = np.log(df['Bookmarks']+0.001)

#     # turn into z scores
    df['Bookmarks'] = df['Bookmarks']/df['Chapters']
    df = df.replace([np.inf, -np.inf], np.nan)
    df[['Chapters','Bookmarks']].dropna()
    df['Bookmarks'] = (df['Bookmarks'] - df['Bookmarks'].mean())/df['Bookmarks'].std(ddof=0)
    
#     df = df.dropna(subset=['Bookmarks'])
#     df = df[np.isfinite(df['Bookmarks'])]
#     df['Bookmarks'] = (df['Bookmarks'] - df['Bookmarks'].mean())/df['Bookmarks'].std(ddof=0)

    return df

In [7]:
def create_dict(df, field, binsize):
    
    # Make bins
    df.JSD = df.JSD.astype(float)
    min_jsd = min(df.JSD.tolist())
    max_jsd = max(df.JSD.tolist())
    dic = {}
    for i in np.arange(min_jsd, max_jsd, binsize):
        df_s = df[(df.JSD >= i) & (df.JSD < i+binsize)]
#         if len(df_s) > 9:
#             print (min_cos, max_cos)
#             print(df_s[field].tolist())
        li = df_s[field].tolist()
        if len(li) > 0:
            dic[i] = bootstrap_resample(li)
    return dic

In [9]:
df_all = []
for i,fandom in enumerate(fandom_list):
    df = pd.read_csv(os.path.join(data_path,fandom), sep = '\t')
    df = create_normed_dfs(df)
    df = df[['JSD', 'Bookmarks']].dropna()
    df_all.append(df)
#         print(fandom, '   ', len(df))
df = pd.concat(df_all)
df = df.sort_values(by = 'JSD')[:-5]
df = df.sort_values(by = 'Bookmarks')[:-5]

dic = create_dict(df, 'Bookmarks', 0.05)

print(dic)


KeyboardInterrupt: 